In [14]:
import json
import random
from itertools import chain
from pathlib import Path
import os
import networkx as nx
import numpy as np
import pandas as pd
import pytest

import pyciemss
from pyciemss.integration_utils.result_processing import convert_to_output_format
from pyciemss.visuals import plots, vega



In [16]:

name = "Straight"
timepoint_list = []
sine_list = []
sample_list = []


def save_result(data, name, ref_ext):
    """Save new reference files"""
    _output_root = 'output'
    mode = "w" if ref_ext == "svg" else "wb"
    with open(os.path.join(_output_root, f"{name}.{ref_ext}"), mode) as f:
        f.write(data)

# create sine fucntions
def return_sine_function_3(function_name):
        if function_name == "Straight":
                a, b, c, d = 0, 0, 0, np.random.randn()
                a2, b2, c2, d2 = 0, 0, 0, 0
        if function_name == "Sine_1a":
                a, b, c, d = 1*(np.random.randn() + 1), .25, np.pi, np.random.randn()
                a2, b2, c2, d2 = 1, .25, np.pi, 0
        if function_name == "Sine_2b":
                a, b, c, d = 2, .25*(np.random.randn() + 1), np.pi, np.random.randn()
                a2, b2, c2, d2 = 2, .25, np.pi, 0
        if function_name == "Sine_3c":
                a, b, c, d = 3, .25, np.pi*(np.random.randn() + 1), np.random.randn()
                a2, b2, c2, d2 = 3, .25, np.pi, 0
        def sine_function(x):
                return np.power(x, .25) + a*np.sin(b*x + c) + d
        def sine_function_default(x):
                return np.power(x, .25) + a2*np.sin(b2*x + c2) + d2
        function_parameters_individual = {'a': a,  'b': b, 'c': c,  'd': d}
        function_parameters_default = {'a': a2, 'b': b2, 'c': c2, 'd': d2}
        
        return sine_function, sine_function_default, function_parameters_individual, function_parameters_default

function_dataframe = pd.DataFrame(columns=['functions', 'sample_id', 'a', 'b',  'c',  'd'])
function_list = []
j = 0
all_functions ={}

# create sine dataframes and save the parameters used
for funct_name in ["Straight", "Sine_1a", "Sine_2b", "Sine_3c"]:
        for _ in range(100):
                j+=1
                timepoint_list.extend([x for x in range(100)])
                sample_list.extend([j]*100)
                random_function,  sine_function_default, function_parameters_individual, function_parameters_default = return_sine_function_3(funct_name)

                function_parameters_individual['functions'] = funct_name
                function_parameters_individual['sample_id'] = j
                function_list.append(function_parameters_individual)
                sine_list.extend([random_function(x) for x in range(100)])
        all_functions[funct_name] = sine_function_default 
        function_parameters_default['functions'] = funct_name + " Default"
        function_parameters_default['sample_id'] = None

#keep track of parameters
function_dataframe = pd.DataFrame(function_list)
function_default_dataframe =  pd.DataFrame(function_parameters_default, index=[0])


sine_distribution= pd.DataFrame({'timepoint_id' : timepoint_list,
                   'sample_id' : sample_list,
                   'Example': sine_list})


traces_list = []   

# get examplary lines
#change kmean here to use clustering
examplary_line = pyciemss.visuals.trajectories.select_traces(sine_distribution, kmean=True)
# get the sample'id for the examplary data used per sample_id and select_by
best_samples_cluster = examplary_line[['sample_id', 'select_by', 'cluster']].drop_duplicates()
best_samples_df_list = []

# for each cluster create a new plot
for cluster in list(np.unique(examplary_line['cluster'])):    
        current_cluster_df = examplary_line[examplary_line['cluster'] == cluster]
        current_cluster_df = current_cluster_df.reset_index()
        # get right input for 
        examplary_lines_pivot = current_cluster_df[['examplary_line', 'mean_trajectory', 'timepoint', 'select_by']].pivot_table(
        values="examplary_line", index = ["timepoint", 'mean_trajectory'], columns="select_by"
        )
        examplary_lines_pivot = examplary_lines_pivot.reset_index()
        examplary_lines_pivot = examplary_lines_pivot.rename(columns = {'mean_trajectory': 'Baseline_Trajectory_Mean'})

        # get baseline for sine functions
        for function_name, function in all_functions.items():
                examplary_lines_pivot["Baseline_" + function_name.title()] = [function(x) for x in range(100)]
        # plot the traces along with the baselines
        schema = plots.trajectories(sine_distribution[['timepoint_id', 'sample_id', 'Example']], traces = examplary_lines_pivot)
        path = "{}_{}".format(name, cluster)
        image = plots.ipy_display(schema, format="PNG").data
        save_result(image, path, "png")
        plots.ipy_display(schema, format="interactive")

# clean up talbe that get the parameters of the samples id's used for examplary
all_clusters = best_samples_cluster.merge(function_dataframe, how="inner", on="sample_id")
all_clusters = pd.concat([all_clusters, function_default_dataframe])[['functions', 'select_by', 'a', 'b', 'c', 'd']]
all_clusters = all_clusters.rename(columns = {'functions': 'Function Name', 'select_by': "Selection Method"})
all_clusters["Function Name"] = all_clusters["Function Name"].apply(lambda x: x.replace("_", " ").title())
all_clusters["Examplar #"] = all_clusters["Selection Method"].apply(lambda x: 1 if len(str(x).split("_")) < 2 else str(x).split("_")[1])
all_clusters["Selection Method"] = all_clusters["Selection Method"].apply(lambda x:  str(x).split("_")[0])
all_clusters = all_clusters.round(2)


all_clusters.to_csv("output/{}.csv".format(name))
# table with sine variables of starting function and the formula of the best select_by lines and metrics1

Init 1


[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:    0.3s
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Do

27.948 --> 

[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done 199 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 449 tasks      | elapsed:    0.1s
[Parallel(n_jobs=1)]: Done 799 tasks      | elapsed:    0.2s
[Parallel(n_jobs=1)]: Done 1249 tasks      | elapsed:    0.4s


18.965 --> 

...

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 7
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

ssr based F test:         F=-7.2000 , p=1.0000  , df_denom=72, df_num=10
ssr based chi2 test:   chi2=-90.0000, p=1.0000  , df=10
likelihood ratio test: chi2=-1667.1593, p=1.0000  , df=10
parameter F test:         F=0.0001  , p=1.0000  , df_denom=72, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.2000 , p=1.0000  , df_denom=72, df_num=10
ssr based chi2 test:   chi2=-90.0000, p=1.0000  , df=10
likelihood ratio test: chi2=-1644.1971, p=1.0000  , df=10
parameter F test:         F=0.0002  , p=1.0000  , df_denom=72, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.2000 , p=1.0000  , df_denom=72, df_num=10
ssr based chi2 test:   chi2=-90.0000, p=1.0000  , df=10
likelihood ratio test: chi2=-1662.2586, p=1.0000  , df=10
parameter F test:         F=0.0001  , p=1.0000  , df_denom=72, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.2000 , p=1.0000  , df_denom=72, df_num=10
ssr based

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

ssr based F test:         F=-7.6998 , p=1.0000  , df_denom=77, df_num=10
ssr based chi2 test:   chi2=-89.9974, p=1.0000  , df=10
likelihood ratio test: chi2=-942.1749, p=1.0000  , df=10
parameter F test:         F=1992.6231, p=0.0000  , df_denom=77, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.6847 , p=1.0000  , df_denom=77, df_num=10
ssr based chi2 test:   chi2=-89.8208, p=1.0000  , df=10
likelihood ratio test: chi2=-559.7356, p=1.0000  , df=10
parameter F test:         F=117744.2761, p=0.0000  , df_denom=77, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.6995 , p=1.0000  , df_denom=77, df_num=10
ssr based chi2 test:   chi2=-89.9944, p=1.0000  , df=10
likelihood ratio test: chi2=-871.5152, p=1.0000  , df=10
parameter F test:         F=4339.5920, p=0.0000  , df_denom=77, df_num=6

Granger Causality
number of lags (no zero) 10
ssr based F test:         F=-7.6990 , p=1.0000  , df_denom=77, df_num=10
ssr bas

/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 10, but rank is 6
  warnings.warn('covariance of constraints does not have full '
/Users/oost464/opt/anaconda3/envs/pyciems310/lib/python3.10/site-packages/statsmodels/base/model.py:1896: ValueWarning: covariance of constraints does not

In [12]:
def distributions():
    model_1_path = (
            "https://raw.githubusercontent.com/DARPA-ASKEM/simulation-integration"
            "/main/data/models/SEIRHD_NPI_Type1_petrinet.json"
    )
    start_time = 0.0
    end_time = 100.0
    logging_step_size = 1
    num_samples = 30
    sample = pyciemss.sample(
        model_1_path,
        end_time,
        logging_step_size,
        num_samples,
        start_time=start_time,
        solver_method="euler",
    )["unprocessed_result"]

    for e in sample.values():
        if len(e.shape) > 1:
            num_timepoints = e.shape[1]

    return convert_to_output_format(
        sample,
        timepoints=np.linspace(start_time, end_time, num_timepoints),
        time_unit="notional",
    )

distributions = distributions()
print(distributions.head())
print(distributions.columns)
all_columns = ["timepoint_id", "sample_id", 'infected_observable_state',
       'exposed_observable_state', 'hospitalized_observable_state',
       'dead_observable_state']
all_columns = ["timepoint_id", "sample_id", 'infected_observable_state']

distributions = distributions[all_columns]
distributions

   timepoint_id  sample_id  persistent_beta_c_param  persistent_kappa_param  \
0             0          0                 0.427889                0.074589   
1             1          0                 0.427889                0.074589   
2             2          0                 0.427889                0.074589   
3             3          0                 0.427889                0.074589   
4             4          0                 0.427889                0.074589   

   persistent_gamma_param  persistent_hosp_param  persistent_death_hosp_param  \
0                0.458113               0.082172                     0.058686   
1                0.458113               0.082172                     0.058686   
2                0.458113               0.082172                     0.058686   
3                0.458113               0.082172                     0.058686   
4                0.458113               0.082172                     0.058686   

   persistent_I0_param   D_state    E_

,timepoint_id,sample_id,infected_observable_state
0,0,0,1.793558e+01
1,1,0,1.749212e+01
2,2,0,1.564300e+01
3,3,0,1.342610e+01
4,4,0,1.127914e+01
...,...,...,...
2965,94,29,1.761435e+06
2966,95,29,1.584844e+06
2967,96,29,1.418121e+06
2968,97,29,1.263510e+06


In [13]:
#'timepoint_id', 'sample_id'
sine_distribution = pd.read_csv("paper_sine_05/paper_sine_05_new_df.csv")
sine_distribution = sine_distribution.rename(
            columns={
                'Unnamed: 0.1': "timepoint_id",
                'Unnamed: 0': "100",
            }
        )
sine_distribution = sine_distribution.melt(id_vars=["timepoint_id"], 
        var_name="sample_id", 
        value_name="same_stat_state")

traces_list = []   
 
#select_by "mean", "var", "granger"
for select_by in ['mean', 'var', 'granger']:
    examplary_line, mean_line_df = pyciemss.visuals.trajectories.select_traces(sine_distribution, select_by = select_by,  kmean=True)
    traces_list.append(examplary_line)


new_df = pd.concat(traces_list, axis = 1)
combined_df = pd.concat([new_df, mean_line_df], axis =1)

schema = plots.trajectories(sine_distribution, traces = combined_df,  keep=".*_state")

plots.save_schema(schema, "_schema.json")

plots.ipy_display(schema, format="interactive")

FileNotFoundError: [Errno 2] No such file or directory: 'paper_sine_05/paper_sine_05_new_df.csv'